# Project Part 2

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/entrylevelcs/CS39AA-Project/blob/main/project_part3.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/entrylevelcs/CS39AA-Project/blob/main/project_part3.ipynb)


## 1. Introduction/Background

For this part of the project we are taking the nearest neighbors model and training it using both our AI and real data sets. From this simple model we can see how well each of the different training data sets do after being optimizing the hyperparameters of this simple model.

## 2. Simple Modeling with KNeighborsClassifier

In [1]:
# import all of the python modules/packages you'll need here
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric
# ...

c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Get the data set from real steam reviews. The specific set of data that I am using for this notebook comes from https://www.kaggle.com/datasets/andrewmvd/steam-reviews/ but is just a sample of 25000 from the entire set.

In [2]:
human_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/human_dataset.csv'
df = pd.read_csv(human_data)
df = df[df["review_text"].notnull()]
sample_size = len(df)
#sample_size = 25000
#df = df.sample(sample_size)
# previous two lines are for sampling from the entire human dataset

Get the data set that was generated by chatgpt. This review data started as only being about CS:GO but has been expanded to be more general and talk about other games.

In [3]:
ai_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/gpt3.5_generated_data.csv'
df2 = pd.read_csv(ai_data)

In [4]:
df = df.rename(columns={"review_text": "text", "review_score": "label"})
df2 = df2.rename(columns={"Review": "text", " Sentiment": "label"})

Sets up our different inputs and outputs. X and y are the inputs and outputs from the real steam reviews while X1 and y1 are from the AI generated reviews. I also adjusted the test size so that the training data size the same length as the AI generated data set. From the shape of each we can see that the vocab size is 11127 words.

In [5]:
ai_data_size = df2.size
sample_size = df.size
proportion = ((sample_size-ai_data_size)/sample_size)

In [6]:
real_data_raw = Dataset.from_pandas(df[['text', 'label']])
simulated_data_raw = Dataset.from_pandas(df2)
#MODEL_NAME = 'gpt2'
MODEL_NAME = 'bert-base-cased'
real_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
simulated_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
realModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, max_length=55, output_attentions=False, output_hidden_states=False)
simulatedModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, max_length=55, output_attentions=False, output_hidden_states=False)


c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN th

In [7]:
def tokenize_function(examples):
    return real_tokenizer(examples["text"], padding='max_length', truncation=True, max_length=55)

real_data = real_data_raw.map(tokenize_function, batched=True)
simulated_data = simulated_data_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/24971 [00:00<?, ? examples/s]

Map:   0%|          | 0/2496 [00:00<?, ? examples/s]

In [8]:
train_prop = proportion
real_data_train = real_data.select(range(int(len(real_data)*train_prop)))
real_data_eval = real_data.select(range(int(len(real_data)*train_prop), len(real_data)))

In [9]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(num_train_epochs=10,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="real",
                                  evaluation_strategy="steps",
                                  eval_steps=200,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

trainer = Trainer(model = realModel, 
                  args = training_args,
                  train_dataset = real_data_train, 
                  eval_dataset = real_data_eval,
                  compute_metrics = compute_metrics,
)

C:\Users\Muajeh Lee\AppData\Local\Temp\ipykernel_10808\3120487532.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [10]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else: 
    device = "cpu"


Using GPU


In [11]:
realModel.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
torch.set_grad_enabled(True)
trainer.train()
trainer.evaluate()

  0%|          | 0/7550 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
